In [ ]:
from __future__ import annotations
from typing import *
if TYPE_CHECKING:
    pass
from numpy.lib.function_base import iterable
from src.Kernel import GaussKernel
from src.EllipsoidRenderer import EllipsoidRenderer
from src.GridBuilder import ImageGridBuilder
from read_data import load_position_data
import numpy as np
import math
import matplotlib.pyplot as plt
import time

plt.gcf().set_dpi(200)

## Load Data

In [ ]:
# Load Data

data = np.load('data/toastgitter/all_data.npy') # Shape [STEPS, NUM_REL]
p_pos, c_pos = load_position_data('data/toastgitter/LOGFILE_unified_coordinates.txt')
NUM_SAMPLES, NUM_RELATIONS = data.shape

print(data.shape)

In [ ]:
# Plot Data

fig, (ax1, ax2) = plt.subplots(2, figsize=(10,12))
ax1.plot(np.mean(data, axis=1))
ax2.plot(np.std(data, axis=1))
plt.show()

## Setup Eperiment

In [ ]:
# Setup experiment

from construct_matrix import MatrixBuilder
mb = MatrixBuilder(
    SPEED_OF_SOUND = 1_484_000.0, # [mm/s]
    SAMPLE_RATE = 100_000_000.0, # [1/s]
    ECHO_START_TIME = 0.0, # [s]

    data_path      = 'data/toastgitter/all_data.npy',
    positions_path = 'data/toastgitter/LOGFILE_unified_coordinates.txt',
    piezo_filter   = lambda pos: np.allclose(pos, np.array([385.0, -110.0, 198.0])),
    cMut_filter    = lambda pos: any(
        np.allclose(pos, np.array([k+311.1, -110.0, 198.0]))
        for k in range(0,61,4)
        for l in range(-5, 7, 5)

    ),
)

u = np.array([40,0,0], dtype=float)
v = np.array([0,0,-20], dtype=float)
o = np.array([320, -110, 10], dtype=float)

res_u = 40
res_v = 20

kernel = GaussKernel(1.0)

In [ ]:
# Plot positions
mb.plot_positions(u=u, v=v, o=o, res_u=res_u, res_v=res_v, kernel=kernel)

In [ ]:
# Construct matrix M

time_start = time.time()
M, lower_sample_idx, upper_sample_idx = mb.build_matrix(u=u, v=v, o=o, res_u=res_u, res_v=res_v, kernel=kernel)
time_stop = time.time()

print(time_stop-time_start)
print(lower_sample_idx, upper_sample_idx)
print(M.shape)

In [ ]:
# Show example row vector of M
fig = plt.imshow(M.reshape([-1,res_u, res_v])[2000], cmap='gray')
plt.colorbar()

# Do Calculations

In [ ]:
# Compute Moore-Penrose-inverse
time_start = time.time()
Minv= np.linalg.pinv(M)
time_stop = time.time()

print(time_stop-time_start)

In [ ]:
# Select necessary data and multiply with inverse matriv
selected_data = mb.data[lower_sample_idx:upper_sample_idx, mb.selected].flatten(order='F')

time_start = time.time()
x_hat = Minv @ selected_data
time_stop = time.time()
print(time_stop-time_start)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(25,5))
ax1.plot(np.std(data, axis=1))
ax1.axvspan(lower_sample_idx, upper_sample_idx, color='red', alpha=0.2)
ax2.imshow(x_hat.reshape([res_u, res_v])[:,::-1].T, cmap='gray')

plt.show()

## Gradient Descent

In [ ]:
# Compute Hessian and M^T @ y
H = M.T @ M
M_y = M.T @ selected_data

In [ ]:
# Normalize
norm_coeff = np.linalg.norm(M_y)

H /= norm_coeff
M_y /= norm_coeff

In [ ]:
plt.gcf().set_dpi(200)
plt.imshow(H)
plt.colorbar()
plt.show()

In [ ]:
# Do calculations
alpha_0 = 0.01
x_grad = M_y.copy()
momentum = np.zeros(x_grad.shape)

grad_norms = []
for it in range(5000):
    alpha = alpha_0 * np.power(it+1, -0.5)
    g = H@x_grad-M_y
    grad_norms.append(np.linalg.norm(g))
    x_grad -= alpha*(g+momentum)
    momentum = g

In [ ]:
# Convergence plot & results

fig, (ax1, ax2) = plt.subplots(1,2, figsize=(20,5))
ax1.plot(grad_norms)
ax2.imshow(x_guess.reshape([res_u, res_v])[:,::-1].T, cmap='gray')
plt.show()

In [ ]:
# Compute determinant of H
np.linalg.det(H)

In [ ]:
diff = x_hat - x_grad
print(np.linalg.norm(diff))

In [ ]:
print('Error of gradient descent method:', np.linalg.norm(M@x_grad-selected_data))
print('Error of moore-penrose method:', np.linalg.norm(M@x_hat-selected_data))

In [ ]:
# Is diff in nullspace of H
np.linalg.norm(H@diff)

## Sparsity Analysis

In [ ]:
threshold = 1e-10
print('Sparsity of M', np.sum(M<threshold)/M.size)
print('Sparsity of M', np.sum(Minv<threshold)/Minv.size)

## Further steps
- plot data interval in date-mean / std plot
- use gpu via torch
- smear out ellips to reduce data dimensionality
- use numba and calculate matrix directly on gpu
- use matrix chunks to handle larger matricies